rendering placeholder

In [ ]:
import rasterio
import matplotlib.pyplot as plt
from pathlib import Path

path = Path("data/roads_terrain_final/train/17428960_15.tiff")
with rasterio.open(path) as src:
    img = src.read()   # shape: (bands, H, W)
    transform = src.transform
    width = src.width
    height = src.height

    # Pixel center indices
    cx = width  / 2
    cy = height / 2

    # Convert pixel center → map coordinates
    center_x, center_y = transform * (cx, cy)

bands, H, W = img.shape

plt.figure(figsize=(5 * bands, 5))
plt.suptitle(f"Image Bands for {path.name}", fontsize=16)
for i in range(bands):
    plt.subplot(1, bands, i+1)
    plt.imshow(img[i], cmap="gray")
    plt.title(f"Band {i+1}")
    plt.axis("off")

plt.show()

In [ ]:
# ==============================================
# Road Detection via Frangi Vesselness
# Full Evaluation Script (100 TIFFs)
# ==============================================

from pathlib import Path
import numpy as np
import rasterio
from tqdm import tqdm
import matplotlib.pyplot as plt

from skimage.filters import frangi
from skimage.morphology import closing, dilation, footprint_rectangle
from skimage.exposure import rescale_intensity
from scipy.ndimage import distance_transform_edt


# ----------------------------------------------
# Metrics
# ----------------------------------------------

def compute_iou(pred_mask, true_mask):
    pred_bin = pred_mask > 0
    true_bin = true_mask > 0
    inter = np.logical_and(pred_bin, true_bin).sum()
    union = np.logical_or(pred_bin, true_bin).sum()
    if union == 0:
        return 1.0 if inter == 0 else 0.0
    return inter / union


def modified_hausdorff_distance(pred_mask, true_mask):
    m1 = pred_mask > 0
    m2 = true_mask > 0

    if not m1.any() and not m2.any():
        return 0.0
    if not m1.any() or not m2.any():
        return 9999.0

    dt2 = distance_transform_edt(~m2)
    d1 = dt2[m1]
    mean_1_to_2 = d1.mean()

    dt1 = distance_transform_edt(~m1)
    d2 = dt1[m2]
    mean_2_to_1 = d2.mean()

    return max(mean_1_to_2, mean_2_to_1)


# ----------------------------------------------
# Frangi vesselness road detector
# ----------------------------------------------

def detect_roads_frangi(rgb):
    gray = np.mean(rgb, axis=0).astype(float)

    # normalize 0–1
    gray = rescale_intensity(gray, out_range=(0, 1))

    # vesselness (roads appear bright)
    vessel = frangi(gray, scale_range=(1, 5), scale_step=2)

    # threshold (90th percentile works well)
    thresh = vessel > np.percentile(vessel, 90)

    # cleanup
    fp = footprint_rectangle((3, 3))
    cleaned = closing(thresh, fp)
    cleaned = dilation(cleaned, fp)

    return cleaned.astype(np.uint8)


# ----------------------------------------------
# Plotting
# ----------------------------------------------

def plot_result(title, rgb, pred, true):
    fig, axs = plt.subplots(1, 4, figsize=(18, 5))

    axs[0].imshow(np.moveaxis(rgb, 0, -1))
    axs[0].set_title(title + " — RGB")
    axs[0].axis("off")

    axs[1].imshow(true, cmap="gray")
    axs[1].set_title("True Mask")
    axs[1].axis("off")

    axs[2].imshow(pred, cmap="gray")
    axs[2].set_title("Predicted (Frangi)")
    axs[2].axis("off")

    overlay = np.moveaxis(rgb, 0, -1).copy()
    overlay[pred > 0] = [255, 0, 0]   # red predicted
    overlay[true > 0] = [0, 255, 0]   # green truth

    axs[3].imshow(overlay)
    axs[3].set_title("Overlay\nRed=Pred, Green=True")
    axs[3].axis("off")

    plt.tight_layout()
    plt.show()


# ----------------------------------------------
# Run evaluation on first 100 TIFFs
# ----------------------------------------------

TIFF_DIR = Path("data/tiffs")

tifs = sorted(TIFF_DIR.glob("*.tif")) + sorted(TIFF_DIR.glob("*.tiff"))
tifs = tifs[:100]

results = []

print("Evaluating Frangi Vesselness on 100 TIFFs...")
for tif in tqdm(tifs, desc="Evaluating"):
    with rasterio.open(tif) as src:
        data = src.read()
        rgb = data[:3]               # first 3 bands are RGB
        true_mask = data[-1]         # last band is road mask

    pred_mask = detect_roads_frangi(rgb)

    iou = compute_iou(pred_mask, true_mask)
    mhd = modified_hausdorff_distance(pred_mask, true_mask)

    results.append({
        "path": tif,
        "iou": iou,
        "mhd": mhd,
        "rgb": rgb,
        "pred": pred_mask,
        "true": true_mask
    })


# ----------------------------------------------
# Summary
# ----------------------------------------------

mean_iou  = np.mean([r["iou"] for r in results])
median_iou = np.median([r["iou"] for r in results])
mean_mhd = np.mean([r["mhd"] for r in results])
median_mhd = np.median([r["mhd"] for r in results])

print("\n=== FRANGI RESULTS ===")
print(f"Mean IoU:   {mean_iou:.4f}")
print(f"Median IoU: {median_iou:.4f}")
print(f"Mean MHD:   {mean_mhd:.2f} px")
print(f"Median MHD: {median_mhd:.2f} px")

# ----------------------------------------------
# Best & Worst
# ----------------------------------------------

best = max(results, key=lambda r: r["iou"])
worst = min(results, key=lambda r: r["iou"])

print("\nBest tile:", best["path"], "IoU =", best["iou"])
print("Worst tile:", worst["path"], "IoU =", worst["iou"])

plot_result("BEST (Frangi)", best["rgb"], best["pred"], best["true"])
plot_result("WORST (Frangi)", worst["rgb"], worst["pred"], worst["true"])